In [1]:
!pip uninstall pandas -y

!pip install -U pandas

!pip uninstall numpy -y

!pip install -U numpy

Found existing installation: pandas 1.3.4
Uninstalling pandas-1.3.4:
  Successfully uninstalled pandas-1.3.4
  Using cached pandas-1.3.4-cp38-cp38-win_amd64.whl (10.2 MB)
Found existing installation: numpy 1.21.3
Uninstalling numpy-1.21.3:
  Successfully uninstalled numpy-1.21.3
  Using cached numpy-1.21.3-cp38-cp38-win_amd64.whl (14.0 MB)


In [17]:
import pandas as pd 
import numpy as np


In [18]:
df = pd.read_csv("loan_prediction.csv")

In [19]:
df.head()

,Unnamed: 0,Income,Age,Experience,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag,skfold,Status
0,0,1303834,23,3,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0,3,single
1,1,7574516,40,10,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0,0,single
2,2,3991815,66,4,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0,3,married
3,3,6256451,41,2,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1,0,single
4,4,5768871,47,11,rented,no,Civil_servant,Tiruchirappalli,Tamil_Nadu,3,14,1,0,single


In [20]:
pip install  xgboost


Note: you may need to restart the kernel to use updated packages.


In [21]:
import xgboost
import sklearn

In [22]:
xgboost.__version__

'1.5.0'

In [25]:
useful_features=[c for c in df.columns if c not in ("Unnamed: 0","Risk_Flag","skfold")]
categorical_feature = [ c for c in df.columns if c in ("House_Ownership","Car_Ownership",
                                                       "Profession","CITY","STATE","Status")]
from sklearn import preprocessing
from xgboost import XGBClassifier 
from sklearn.metrics import confusion_matrix,accuracy_score  

final_predictions = []
scores = []
for fold in range(5):
    xtrain=df[df.skfold!= fold].reset_index(drop=True)
    xvalid=df[df.skfold==fold].reset_index(drop=True)
    
    ytrain=xtrain.Risk_Flag
    yvalid=xvalid.Risk_Flag
    xtrain=xtrain[useful_features]
    xvalid=xvalid[useful_features]
    

    #OHE with pandas 
    xtrain_ohe=pd.get_dummies(xtrain[categorical_feature])
    xvalid_one=pd.get_dummies(xvalid[categorical_feature])
    
    xtrain=pd.concat([xtrain,xtrain_ohe],axis=1)
    xvalid=pd.concat([xvalid,xvalid_one],axis=1)
    
      #i'm droping categorical columns
    xtrain = xtrain.drop(categorical_feature, axis=1)
    xvalid = xvalid.drop(categorical_feature, axis=1)
    
    
    scaler = preprocessing.MinMaxScaler()
    xtrain= scaler.fit_transform(xtrain)
    xvalid = scaler.transform(xvalid)
    xtrain= pd.DataFrame(xtrain)
    xvalid = pd.DataFrame(xvalid)
    print(xtrain.shape)
    model = XGBClassifier(random_state=42,
                          n_jobs=-1,
                          learning_rate=.10602386982172891,
                          max_depth=10,
                          reg_lambda=.3186871669016709,
                          reg_alpha=.0016827056276966232,
                          subsample=.9116197200738823,
                          colsample_bytree=.3564391606885141,
                          n_estimators=8000)
    model.fit(xtrain, ytrain,early_stopping_rounds=300,eval_set=[(xvalid,yvalid)])
    preds_valid = model.predict(xvalid)
   # test_preds = model.predict(xtest)
    #final_predictions.append(test_preds)
    ac = accuracy_score(yvalid, preds_valid)
    print(fold, ac)
    scores.append(ac)

print(np.mean(scores), np.std(scores))

(201600, 407)


C:\Users\Debjit\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:14:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.63499
[1]	validation_0-logloss:0.58775
[2]	validation_0-logloss:0.54882
[3]	validation_0-logloss:0.51690
[4]	validation_0-logloss:0.49081
[5]	validation_0-logloss:0.46880
[6]	validation_0-logloss:0.45002
[7]	validation_0-logloss:0.43417
[8]	validation_0-logloss:0.42143
[9]	validation_0-logloss:0.41034
[10]	validation_0-logloss:0.40141
[11]	validation_0-logloss:0.39342
[12]	validation_0-logloss:0.38639
[13]	validation_0-logloss:0.38028
[14]	validation_0-logloss:0.37419
[15]	validation_0-logloss:0.36976
[16]	validation_0-logloss:0.36645
[17]	validation_0-logloss:0.36281
[18]	validation_0-logloss:0.35913
[19]	validation_0-logloss:0.35573
[20]	validation_0-lo

[229]	validation_0-logloss:0.26795
[230]	validation_0-logloss:0.26777
[231]	validation_0-logloss:0.26766
[232]	validation_0-logloss:0.26746
[233]	validation_0-logloss:0.26729
[234]	validation_0-logloss:0.26726
[235]	validation_0-logloss:0.26689
[236]	validation_0-logloss:0.26681
[237]	validation_0-logloss:0.26667
[238]	validation_0-logloss:0.26644
[239]	validation_0-logloss:0.26632
[240]	validation_0-logloss:0.26617
[241]	validation_0-logloss:0.26596
[242]	validation_0-logloss:0.26587
[243]	validation_0-logloss:0.26569
[244]	validation_0-logloss:0.26531
[245]	validation_0-logloss:0.26522
[246]	validation_0-logloss:0.26506
[247]	validation_0-logloss:0.26486
[248]	validation_0-logloss:0.26469
[249]	validation_0-logloss:0.26457
[250]	validation_0-logloss:0.26440
[251]	validation_0-logloss:0.26409
[252]	validation_0-logloss:0.26387
[253]	validation_0-logloss:0.26377
[254]	validation_0-logloss:0.26350
[255]	validation_0-logloss:0.26335
[256]	validation_0-logloss:0.26330
[257]	validation_0-l

[464]	validation_0-logloss:0.23492
[465]	validation_0-logloss:0.23480
[466]	validation_0-logloss:0.23460
[467]	validation_0-logloss:0.23454
[468]	validation_0-logloss:0.23439
[469]	validation_0-logloss:0.23426
[470]	validation_0-logloss:0.23418
[471]	validation_0-logloss:0.23413
[472]	validation_0-logloss:0.23408
[473]	validation_0-logloss:0.23396
[474]	validation_0-logloss:0.23381
[475]	validation_0-logloss:0.23374
[476]	validation_0-logloss:0.23362
[477]	validation_0-logloss:0.23351
[478]	validation_0-logloss:0.23345
[479]	validation_0-logloss:0.23336
[480]	validation_0-logloss:0.23323
[481]	validation_0-logloss:0.23321
[482]	validation_0-logloss:0.23316
[483]	validation_0-logloss:0.23305
[484]	validation_0-logloss:0.23300
[485]	validation_0-logloss:0.23288
[486]	validation_0-logloss:0.23279
[487]	validation_0-logloss:0.23265
[488]	validation_0-logloss:0.23254
[489]	validation_0-logloss:0.23239
[490]	validation_0-logloss:0.23235
[491]	validation_0-logloss:0.23233
[492]	validation_0-l

...

[934]	validation_0-logloss:0.21663
[935]	validation_0-logloss:0.21662
[936]	validation_0-logloss:0.21661
[937]	validation_0-logloss:0.21658
[938]	validation_0-logloss:0.21658
[939]	validation_0-logloss:0.21657
[940]	validation_0-logloss:0.21656
[941]	validation_0-logloss:0.21653
[942]	validation_0-logloss:0.21651
[943]	validation_0-logloss:0.21651
[944]	validation_0-logloss:0.21648
[945]	validation_0-logloss:0.21647
[946]	validation_0-logloss:0.21646
[947]	validation_0-logloss:0.21642
[948]	validation_0-logloss:0.21641
[949]	validation_0-logloss:0.21640
[950]	validation_0-logloss:0.21640
[951]	validation_0-logloss:0.21640
[952]	validation_0-logloss:0.21637
[953]	validation_0-logloss:0.21638
[954]	validation_0-logloss:0.21637
[955]	validation_0-logloss:0.21636
[956]	validation_0-logloss:0.21636
[957]	validation_0-logloss:0.21635
[958]	validation_0-logloss:0.21634
[959]	validation_0-logloss:0.21634
[960]	validation_0-logloss:0.21631
[961]	validation_0-logloss:0.21631
[962]	validation_0-l

[1164]	validation_0-logloss:0.21429
[1165]	validation_0-logloss:0.21429
[1166]	validation_0-logloss:0.21427
[1167]	validation_0-logloss:0.21426
[1168]	validation_0-logloss:0.21424
[1169]	validation_0-logloss:0.21424
[1170]	validation_0-logloss:0.21424
[1171]	validation_0-logloss:0.21421
[1172]	validation_0-logloss:0.21420
[1173]	validation_0-logloss:0.21419
[1174]	validation_0-logloss:0.21420
[1175]	validation_0-logloss:0.21419
[1176]	validation_0-logloss:0.21419
[1177]	validation_0-logloss:0.21419
[1178]	validation_0-logloss:0.21419
[1179]	validation_0-logloss:0.21419
[1180]	validation_0-logloss:0.21420
[1181]	validation_0-logloss:0.21421
[1182]	validation_0-logloss:0.21418
[1183]	validation_0-logloss:0.21416
[1184]	validation_0-logloss:0.21417
[1185]	validation_0-logloss:0.21416
[1186]	validation_0-logloss:0.21415
[1187]	validation_0-logloss:0.21415
[1188]	validation_0-logloss:0.21413
[1189]	validation_0-logloss:0.21414
[1190]	validation_0-logloss:0.21413
[1191]	validation_0-logloss:

[1392]	validation_0-logloss:0.21356
[1393]	validation_0-logloss:0.21357
[1394]	validation_0-logloss:0.21357
[1395]	validation_0-logloss:0.21358
[1396]	validation_0-logloss:0.21358
[1397]	validation_0-logloss:0.21359
[1398]	validation_0-logloss:0.21359
[1399]	validation_0-logloss:0.21358
[1400]	validation_0-logloss:0.21357
[1401]	validation_0-logloss:0.21356
[1402]	validation_0-logloss:0.21356
[1403]	validation_0-logloss:0.21355
[1404]	validation_0-logloss:0.21356
[1405]	validation_0-logloss:0.21355
[1406]	validation_0-logloss:0.21355
[1407]	validation_0-logloss:0.21355
[1408]	validation_0-logloss:0.21355
[1409]	validation_0-logloss:0.21354
[1410]	validation_0-logloss:0.21354
[1411]	validation_0-logloss:0.21356
[1412]	validation_0-logloss:0.21355
[1413]	validation_0-logloss:0.21355
[1414]	validation_0-logloss:0.21354
[1415]	validation_0-logloss:0.21353
[1416]	validation_0-logloss:0.21353
[1417]	validation_0-logloss:0.21354
[1418]	validation_0-logloss:0.21354
[1419]	validation_0-logloss:

[1620]	validation_0-logloss:0.21328
[1621]	validation_0-logloss:0.21329
[1622]	validation_0-logloss:0.21328
[1623]	validation_0-logloss:0.21327
[1624]	validation_0-logloss:0.21327
[1625]	validation_0-logloss:0.21326
[1626]	validation_0-logloss:0.21327
[1627]	validation_0-logloss:0.21326
[1628]	validation_0-logloss:0.21326
[1629]	validation_0-logloss:0.21326
[1630]	validation_0-logloss:0.21326
[1631]	validation_0-logloss:0.21326
[1632]	validation_0-logloss:0.21328
[1633]	validation_0-logloss:0.21328
[1634]	validation_0-logloss:0.21328
[1635]	validation_0-logloss:0.21329
[1636]	validation_0-logloss:0.21328
[1637]	validation_0-logloss:0.21327
[1638]	validation_0-logloss:0.21327
[1639]	validation_0-logloss:0.21327
[1640]	validation_0-logloss:0.21326
[1641]	validation_0-logloss:0.21327
[1642]	validation_0-logloss:0.21327
[1643]	validation_0-logloss:0.21327
[1644]	validation_0-logloss:0.21328
[1645]	validation_0-logloss:0.21329
[1646]	validation_0-logloss:0.21330
[1647]	validation_0-logloss:

[1848]	validation_0-logloss:0.21365
[1849]	validation_0-logloss:0.21364
[1850]	validation_0-logloss:0.21364
[1851]	validation_0-logloss:0.21364
[1852]	validation_0-logloss:0.21366
[1853]	validation_0-logloss:0.21364
[1854]	validation_0-logloss:0.21364
[1855]	validation_0-logloss:0.21364
[1856]	validation_0-logloss:0.21364
[1857]	validation_0-logloss:0.21365
[1858]	validation_0-logloss:0.21364
[1859]	validation_0-logloss:0.21363
[1860]	validation_0-logloss:0.21364
[1861]	validation_0-logloss:0.21364
[1862]	validation_0-logloss:0.21364
[1863]	validation_0-logloss:0.21363
[1864]	validation_0-logloss:0.21363
[1865]	validation_0-logloss:0.21364
[1866]	validation_0-logloss:0.21364
[1867]	validation_0-logloss:0.21366
[1868]	validation_0-logloss:0.21365
[1869]	validation_0-logloss:0.21365
[1870]	validation_0-logloss:0.21365
[1871]	validation_0-logloss:0.21367
[1872]	validation_0-logloss:0.21367
[1873]	validation_0-logloss:0.21368
[1874]	validation_0-logloss:0.21368
[1875]	validation_0-logloss:

In [26]:
import pickle
pickle_out =open("model.pkl","wb")
pickle.dump(model,pickle_out)
pickle_out.close()

In [27]:
load_model = pickle.load(open('model.pkl', 'rb'))

In [28]:
model_score_r1 = load_model.predict(xvalid)
accuracy_score(yvalid, preds_valid)

0.8992857142857142